In [3]:
import numpy as np
import pandas as pd
import os
import sys
#!conda install --yes --prefix {sys.prefix} scikit-learn
!ls
os.listdir()

Untitled.ipynb        rfc_predictions.csv   test.csv
f_rfr.csv             rfr_norm.csv          train.csv
gender_submission.csv rfr_predictions.csv


['.DS_Store',
 'rfr_norm.csv',
 'Untitled.ipynb',
 'rfc_predictions.csv',
 'f_rfr.csv',
 'test.csv',
 'train.csv',
 '.ipynb_checkpoints',
 'rfr_predictions.csv',
 'gender_submission.csv']

In [4]:
# Loading dataset
train_path = 'train.csv'
test_path = 'test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
features_tr = pd.get_dummies(train_data[['Pclass', 'Sex', 'Parch']], columns=['Pclass', 'Sex', 'Parch'])
features_te = pd.get_dummies(test_data[['Pclass', 'Sex', 'Parch']], columns=['Pclass', 'Sex', 'Parch'])

print(len(features_te.columns), len(features_tr.columns))
# > Different Parch value for test

print('Test:', features_te.columns)
print('Train:', features_tr.columns)
# Parch 9 must be deleted (Note: This makes our prediction worse, problem here is too small tr set)

13 12
Test: Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Parch_0',
       'Parch_1', 'Parch_2', 'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6',
       'Parch_9'],
      dtype='object')
Train: Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Parch_0',
       'Parch_1', 'Parch_2', 'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6'],
      dtype='object')


In [7]:
# Manually delete Parch
features_te = features_te.drop(labels='Parch_9', axis=1)
print('Test:', features_te.columns)
print('Train:', features_tr.columns)

Test: Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Parch_0',
       'Parch_1', 'Parch_2', 'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6'],
      dtype='object')
Train: Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Parch_0',
       'Parch_1', 'Parch_2', 'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6'],
      dtype='object')


In [8]:
# Train RF Algorithm
# Trying both RFclassifier & RFregressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

X = features_tr
y = train_data['Survived']

# RFC Model
model_rfc = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt')

# Fit RFC
model_rfc.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
X_test = features_te
rfc_predictions = model_rfc.predict(X_test)

# Turn into df & Save as csv
predictions_rfc = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': rfc_predictions})
predictions_rfc.to_csv('rfc_predictions.csv', index=False)
predictions_rfc.head()
# Score: 75.59

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [10]:
# Try out RF Regressor & Round values

# Max_dept = 4 as only 3 parameters
model_rfr = RandomForestRegressor(n_estimators=100, max_depth=4, max_features='sqrt')

model_rfr.fit(X, y)

rfr_predictions = model_rfr.predict(X_test).round().astype(int)
print(rfr_predictions[:10])
predictions_rfr = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': rfr_predictions})
predictions_rfr.to_csv('rfr_predictions.csv', index=False)
predictions_rfr.head()
# Score: 76.076

[0 1 0 0 0 0 1 0 1 0]


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [27]:
# Notes: 
# I think sqrt sucks for the few features I take
# Do I need dummies for RF? 
# Without OH-encoding

f_tr = train_data[['Pclass', 'Sex', 'Parch']].copy()
f_te = test_data[['Pclass', 'Sex', 'Parch']].copy()


# Turn male into 1 & female into 0
f_tr.loc[f_tr['Sex'] == 'male', 'Sex'] = 1
f_tr.loc[f_tr['Sex'] == 'female', 'Sex'] = 0

f_te.loc[f_te['Sex'] == 'male', 'Sex'] = 1
f_te.loc[f_te['Sex'] == 'female', 'Sex'] = 0

X = f_tr
y = train_data['Survived']
X_test = f_te

mod_rfc = RandomForestRegressor(n_estimators=100, bootstrap=True, max_features='sqrt')
mod_rfc.fit(X, y)

pred_rfc = mod_rfc.predict(X_test).round().astype(int)

fin_pred = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': pred_rfc})
fin_pred.to_csv('rfr_norm.csv', index=False)
# Score: 77.51

In [381]:
# Back to starters; Splitting up the train set to be able to test it w/o wifi
import copy

# 891 entries in train_data
split = round(0.9*891)

t_set = train_data[:split].copy()
t_sol = train_data[:split]['Survived'].copy()
v_set = train_data[split:].copy()
v_sol = train_data[split:]['Survived'].copy()

# Full Training for submission
f_set = train_data.copy()
f_sol = train_data['Survived'].copy()
s_set = test_data.copy()

# Compute age average to replace NaN vaues wth it ; Median has ~same perf
age_avg = train_data['Age'].mean().astype(int)

# Encode Male/Female to 1 & 0 ; Encode Embarked to 2, 1, 0
# Swapping out NaN values
# Name & Ticket Nr should not make a difference

# print(i.dtypes)
# print(train_data.loc[train_data['Embarked'].isnull(), 'Embarked'])
# print(train_data.loc[829]) 

for i in [t_set, v_set, f_set, s_set]:
    i.loc[i['Sex'] == 'male', 'Sex'] = 1
    i.loc[i['Sex'] == 'female', 'Sex'] = 0

    i.loc[i['Embarked'] == 'C', 'Embarked'] = 2
    i.loc[i['Embarked'] == 'Q', 'Embarked'] = 1
    i.loc[i['Embarked'] == 'S', 'Embarked'] = 0

    i.loc[i['Age'].isnull(), 'Age'] = age_avg  
    i.loc[i['Embarked'].isnull(), 'Embarked'] = 0  
    i.loc[i['Fare'].isnull(), 'Fare'] = 0  
    
    i['Fare'] = np.float32(i['Fare'])
    i['Age'] = np.float32(i['Age'])
 
    
# Defining Regressor Model
# Not limiting max_depth & max_features with sqrt seems better
m_rfr = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, max_features=None)
m_rfr.fit(t_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']], t_sol)
# Test Regressor Model
p_rfr = m_rfr.predict(v_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]).round().astype(int)

# Calculate Accuracy by getting the diff in an np array: -1/1 if wrong; 0 if correct
diff = p_rfr - np.array(v_sol)
acc = 1 - (sum(np.abs(diff)) / len(diff))
print(acc)

# Full model & sub
#m_rfr.fit(f_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']], f_sol)
#pf = m_rfr.predict(s_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]).round().astype(int)
#f_pred = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': pf})
#f_pred.to_csv('f_rfr.csv', index=False) # f for full referring to using the full spectrum of data

# 76.55 Final Acc

0.8426966292134832


In [382]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

In [383]:
# Implementing a NN – Do it once with torch, once self-coded

class Network(nn.Module):
    def __init__(self, features):
        super().__init__()
        
        # F functions
        self.fc1 = nn.Linear(features, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        
        
    def forward(self, x):
        
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = F.sigmoid(x)
        
        return x

In [384]:
# Hyperparameters
n_inputs = 6
lr = 0.01
betas = (0.9, 0.999)
epochs = 100
# Not using mini_batches, but full batches here, since not much data anyways


m_nn = Network(features = n_inputs)
m_nn

Network(
  (fc1): Linear(in_features=6, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [385]:
criterion = nn.MSELoss()
optimizer = optim.Adam(m_nn.parameters(), lr=lr, betas=betas)

In [386]:
def train(model, criterion, optimizer, train_X, train_y, test_X, test_y, epochs=1):

    for i in range(epochs):
        
        train_loss = 0
        test_loss = 0
        
        # Train
        
        model.train()
        
        out = model(train_X)
        
        out = out.squeeze()
        loss = (criterion(out, train_y))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Evaluate
            
        model.eval()

        out = model(test_X.float())
        loss = criterion(out, test_y.float())

        test_loss += loss.item()   
        
        # Accuracy
        pred = np.round(out.detach().numpy())
        test_y_np = test_y.numpy().reshape(-1, 1)
        diff = np.abs(pred - test_y_np)
        acc = 1 - (np.sum(diff) / len(diff))             
        
        if (i + 1) % 10 == 0: 
            print('Epoch: {}/{} \t Train Loss: {} \t Test Loss: {} \t Accuracy: {}'.format(i+1, 
                epochs, round(train_loss, 5), round(test_loss, 5), round(acc, 5)))
        
    return model 
        

In [387]:
arr = np.array(t_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']])
arr = np.float32(arr)
train_X = torch.tensor(arr, dtype=torch.float)
print(type(train_X))

arr = np.array(t_sol)
train_y = torch.tensor(arr, dtype=torch.float)

arr = np.array(v_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']])
arr = np.float32(arr)
test_X = torch.tensor(arr, dtype=torch.float)

arr = np.array(v_sol)
test_y = torch.tensor(arr, dtype=torch.float)
    
tr_m = train(m_nn, criterion, optimizer, train_X, train_y, test_X, test_y, epochs=epochs)

<class 'torch.Tensor'>
Epoch: 10/100 	 Train Loss: 0.25748 	 Test Loss: 0.24427 	 Accuracy: 0.62921


/Users/niklasmunnighoff/miniconda3/envs/titanic/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/niklasmunnighoff/miniconda3/envs/titanic/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([89])) that is different to the input size (torch.Size([89, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 20/100 	 Train Loss: 0.24898 	 Test Loss: 0.24326 	 Accuracy: 0.62921
Epoch: 30/100 	 Train Loss: 0.2338 	 Test Loss: 0.23498 	 Accuracy: 0.62921
Epoch: 40/100 	 Train Loss: 0.21118 	 Test Loss: 0.27489 	 Accuracy: 0.70787
Epoch: 50/100 	 Train Loss: 0.20539 	 Test Loss: 0.24553 	 Accuracy: 0.62921
Epoch: 60/100 	 Train Loss: 0.19564 	 Test Loss: 0.26396 	 Accuracy: 0.62921
Epoch: 70/100 	 Train Loss: 0.17526 	 Test Loss: 0.27648 	 Accuracy: 0.66292
Epoch: 80/100 	 Train Loss: 0.17137 	 Test Loss: 0.29317 	 Accuracy: 0.73034
Epoch: 90/100 	 Train Loss: 0.15843 	 Test Loss: 0.34133 	 Accuracy: 0.83146
Epoch: 100/100 	 Train Loss: 0.17934 	 Test Loss: 0.29431 	 Accuracy: 0.7191


In [388]:
# Even though performance is bad, predict & submit

arr = np.array(s_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']])
arr = np.float32(arr)
s_test = torch.tensor(arr, dtype=torch.float)

s_pred = tr_m(s_test)
s_pred = np.round(s_pred.squeeze().detach().numpy()).astype(int)
s_pred = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': s_pred})
s_pred.to_csv('nn.csv', index=False)

# 0.70813

In [406]:
# Creating a NN from scratch

def init_parameters(l_dims):
    '''
    Initializes parameters for NN.
    
    Args:
    l_dims: features of each layer (incl. X layer)

    Returns:
    params: Dict w/ Weight & Bias matrices; Shapes: W: n[next] x n[previous] / B: n[next] x 1
    '''
    params = {}
    
    for i in range(1, len(l_dims)):
        
        params['W' + str(i)] = np.random.randn(l_dims[i], l_dims[i-1])
        params['B' + str(i)] = np.random.randn(l_dims[i], 1)
    
    return params

def L_model_forward(X, params):
    '''
    Propagates forward in network.
    
    Args:
    X: Input matrix; Shape: nx (features) x m 
    parameters: Dict w/ W & B matrices; Shapes: W: n[next] x n[previous] / B: n[next] x 1
    
    Returns:
    Y_hat: Output; Shape: 1 x m
    caches: Values needed for backprop
    '''
    caches = []
    A = X
    L = len(params) // 2 # This is the amount of layers + 1 (X layer does not count!)
    
                                                          
    for i in range(1, L):
        A_prev = A
        A, cache = lin_forward(A_prev, params['W' + str(i)], params['B' + str(i)], 'relu')
        caches.append(cache)
    
    A_prev = A
    A, cache = lin_forward(A_prev, params['W' + str(L)], params['B' + str(L)], 'sigmoid')
    caches.append(cache)
    
    return A, caches
        

def lin_forward(A_prev, W, B, activation):
    '''
    Performs one step forward
    
    Args:
    A_prev: Last activation; Shape: n_prev(==features), m
    W: Weights Matrix for next layer; Shape: n[next] x n[previous]
    B: Bias Matrix for next layer; Shape: n[next], 1
    activation: String for activation we want to execute
    
    Returns:
    A: Next activation; Shape: n[next] x m  
    cache: dict w/ Z for later backprop; Z Shape: n[next] x m
        (Note for A & Z n[next] are the n of that layer itself, hence they will turn to n_prev for the next layer)
    '''
    
    Z = np.dot(W, A_prev) + B
    linear_cache = (A_prev, W, B)
    
    if activation == 'relu':
        A = np.maximum(0, Z)  
        
    elif activation =='sigmoid':
        A = 1/(1 - np.exp(-Z))
        
    elif activation == 'leaky_relu':
        Z[Z<0] = 0.01 * Z[Z<0]
        A = Z
    
    activation_cache = Z
    cache = (linear_cache, activation_cache)

    return A, cache
    
        
def calc_loss(Y_hat, Y):
    '''
    Calculates loss of NN.
    
    Args:
    Y_hat: Prediction; Shape: 1 x m
    Y: Reality; Shape: 1 x m
    
    Returns:
    L: L1 Loss; float
    '''
    m = Y.shape[1]
    loss = 1/m * np.sum(0.5 * np.square(np.abs(Y_hat - Y)))
    return loss
    
def L_model_backward(Y_hat, Y, caches):
    '''
    Backward propagates and calculates attributable losses via Deltas.
    
    Args:
    Y_hat: The predictions of our model; Shape: 1 x m
    Y: The reality; Shape: 1 x m
    caches: Dictionary of stored values; Contains activation (Z) & linear caches (A_prev, W, B)
    
    Returns:
    grads: Dictionary w/ dW & dB values; Shape: dW: n[next] x n[prev] / dB: n[next] x 1
    '''
    grads = {}
    L = len(caches)
    m = Y.shape[1]
    
    # Our L function was: Sum over m of |Y-hat - Y|
    # dL/dZ = dL/dY_hat * dY_hat/dZ
    dY_hat = 1/m * (Y_hat - Y)
    cur_cache = caches[L-1] # This contains our Z
    dA, dW, dB = lin_backward(dY_hat, cur_cache, 'sigmoid')
    grads['dA' + str(L)], grads['dW' + str(L)], grads['dB' + str(L)] = dA, dW, dB
    
    # Loop from L-2 to 0; For 3 layers loop is: 1, 0; Hence +1 when storing
    # We only loop over 2/3 layers, as one is done manually before
    for i in reversed(range(L-1)):
        cur_cache = caches[i]
        dA_prev, dW, dB = lin_backward(grads['dA' + str(i+2)], cur_cache, 'leaky_relu')
        grads['dA' + str(i+1)], grads['dW' + str(i+1)], grads['dB' + str(i+1)] = dA_prev, dW, dB
        # We're storing the previous dA together with weights & biases of the next layer here
        # i.e. the dA we store ets multiplied by the weights with it to arrive at the next Z's
    return grads
        
def lin_backward(dA, cache, activation):
    '''
    Performs one step backward
    
    Args:
    dA: The derivative of the last activation starting with dY_hat; Shape: n_prev(==features), m
    cache: Contains the forward linear & activation parameters
    activation: String describing the activation done on the forward step
    
    Returns:
    dA_prev: The derivatve of the next A going backwards (This will be fed in again in lin_backward); Shape: n_prev, m
    dW: The derivative of W in respect to our loss; Shape: n[next] x n [prev]
    dB: The derivative of B in respect to our loss; Shape: n[next] x 1
    '''
    linear_cache, activation_cache = cache
    
    Z = activation_cache
    
    if activation == 'relu':
        dZ = np.array(dA, copy=True) 
        dZ[Z <= 0] = 0
        dZ[Z > 0] = 1

    elif activation =='sigmoid':
        s = 1/(1 - np.exp(-Z))
        dZ = dA * s * (1 - s) # Element wise multiplication, as we are multiplying the chain derivatives dL/dA * dA/dZ
    
    elif activation == 'leaky_relu':
        dZ = np.array(dA, copy=True) 
        dZ[Z <= 0] = 0.01
        dZ[Z > 0] = 1
        
        
    A_prev, W, B = linear_cache
    m = A_prev.shape[1]
    #print(dZ.shape, A_prev.shape)
    dW = 1/m * np.dot(dZ, A_prev.T)
    dB = 1/m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, dB
    
    
def update_parameters(params, grads, lr):
    '''
    Takes current cache & updates W & B
    
    Args:
    params: Dict containing our current parameters; W & B
    grads: Dict containing the grads; dW & dB 
    lr: Learning rate
    
    Returns:
    params: Dict containing updated parameters; W & B
    '''
    L = len(params) // 2
    
    for l in range(1, L+1):
        #print(grads['dW' + str(l)].shape,  params['W' + str(l)].shape)
        params['W' + str(l)] -= lr * grads['dW' + str(l)]
        params['B' + str(l)] -= lr * grads['dB' + str(l)] 
        
    return params

def get_batch(X, Y, j, mb_size):
    '''
    Returns a mini batch given an mb size
    
    Args:
    X: Inputs; Shape: nx x m
    Y: Outputs; Shape: 1 x m
    j: jth minibatch
    '''
    return X[:, j*mb_size:(j+1)*mb_size], Y[:, j*mb_size:(j+1)*mb_size]


In [413]:
def L_layer_model(X, Y, test_X, l_dims=[4, 2, 1], lr=0.001, epochs=10, print_cost=True, mb_size=32):
    '''
    Trains an L-layer model with ReLU's & a final sigmoid.
    
    Args:
    X: Input features; shape: nx x m
    Y: Outputs; shape: 1 x m
    l_dims: Features of each layer (must include X layer features as first no)
    lr: Learning rate
    epochs: Epochs to train
    
    Returns:
    model: Trained model
    '''
    costs = []
    params = init_parameters(l_dims)
    m = X.shape[1]
    
    for i in range(0, epochs+1):
        
        running_loss = 0
        
        for j in range(0, m // mb_size):
            
            X_batch, Y_batch = get_batch(X, Y, j, mb_size)
        
            Y_hat, caches = L_model_forward(X_batch, params)
            loss = calc_loss(Y_hat, Y_batch)
            grads = L_model_backward(Y_hat, Y_batch, caches)
            params = update_parameters(params, grads, lr)
            
            running_loss += loss
        
        if print_cost and i % 10 == 0:
            print('Epoch: {}/{} \t Cost: {}'.format(i, epochs, running_loss))
    
    # Calculating Accuracy
    Y_hat, _ = L_model_forward(X, params)
    Y_hat = np.round(Y_hat)
    diff = np.abs(Y_hat - Y)
    acc = 1 - (np.sum(diff) / diff.shape[1])   
    
    print('Final Accuracy on Test Set: {}'.format(acc))
    
    Y_hat, _ = L_model_forward(test_X, params)
    Y_hat = np.round(Y_hat)
    Y_hat = np.squeeze(Y_hat, axis=0).astype(int)
    
    man_pred = pd.DataFrame(data={'PassengerId': test_data['PassengerId'], 'Survived': Y_hat})
    man_pred.to_csv('man_nn.csv', index=False)
    
            
    return params
        
        

In [414]:
train_X = np.array(f_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']], dtype=np.float32).T
test_X = np.array(s_set[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']], dtype=np.float32).T

train_y = np.array(f_sol, dtype=np.float32)
train_y = np.expand_dims(train_y, axis=0)

print(train_X.shape, train_y.shape)

(6, 891) (1, 891)


In [415]:
X = train_X
Y = train_y

# Hyperparams
epochs = 500
lr = 0.001
l_dims = [train_X.shape[0], 256, 128, 64, 1]

params_t = L_layer_model(X, Y, test_X, l_dims, lr, epochs)

/Users/niklasmunnighoff/miniconda3/envs/titanic/lib/python3.7/site-packages/ipykernel_launcher.py:74: RuntimeWarning: overflow encountered in exp
/Users/niklasmunnighoff/miniconda3/envs/titanic/lib/python3.7/site-packages/ipykernel_launcher.py:158: RuntimeWarning: overflow encountered in exp


Epoch: 0/500 	 Cost: 16.337861314228935
Epoch: 10/500 	 Cost: 5.1970451853638835
Epoch: 20/500 	 Cost: 5.192869813528542
Epoch: 30/500 	 Cost: 5.193470582239442
Epoch: 40/500 	 Cost: 5.192972419089142
Epoch: 50/500 	 Cost: 5.190236808372639
Epoch: 60/500 	 Cost: 5.1915024059846875
Epoch: 70/500 	 Cost: 5.192670800688336
Epoch: 80/500 	 Cost: 5.189104208930733
Epoch: 90/500 	 Cost: 5.187500028428315
Epoch: 100/500 	 Cost: 5.1875
Epoch: 110/500 	 Cost: 5.1875
Epoch: 120/500 	 Cost: 5.187500000005828
Epoch: 130/500 	 Cost: 5.1875
Epoch: 140/500 	 Cost: 5.194779834170646
Epoch: 150/500 	 Cost: 5.193837904211741
Epoch: 160/500 	 Cost: 5.193514068058348
Epoch: 170/500 	 Cost: 5.193514056725555
Epoch: 180/500 	 Cost: 5.193514045392809
Epoch: 190/500 	 Cost: 5.193514034060105
Epoch: 200/500 	 Cost: 5.193514022727444
Epoch: 210/500 	 Cost: 5.193514011394824
Epoch: 220/500 	 Cost: 5.193514000062247
Epoch: 230/500 	 Cost: 5.193513988729714
Epoch: 240/500 	 Cost: 5.193513977398016
Epoch: 250/500 	

In [372]:
# Logbook [Change ; Results]

# MiniBatches ; 100 Epochs: ~532.1 Cost
# dZ[>0] ; 100 Epochs: ~536 cost, but steadier dec.
# LeakyRELU ; 100 Epochs: 537.5 cost; 1000: 532.6
# From L1 to L2 Loss (Y_H - Y & 1 to 0.5 * (Y_H - Y)^2 & )

# Accuracy always the exact same
# > It seems like the LR was to high as tiny changes in data matter or not
# It seems like there is a maximum at Cost 532, which the algo is not able to surpass


# maybe the sigmoid?  Too high & to low vals? replace with linear layer ?
# All values seem to have the same decimals after comma 
# Weights seem to be exploding very quickly to high minus values


# References:
# https://www.kaggle.com/ktattan/3-layer-neural-network-on-titanic-data

1.0
